In [2]:
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn

In [3]:
names=open('yob2004.txt').read().splitlines()
for i in range(len(names)):
    s=""
    for j in names[i].lower():
        if j==",":
            names[i]=s
            break
        else:
            s+=j
s=""
for i in names:
    for j in i:
        s+=j
s=['.']+sorted(set(s))


# Creating A Dictionary For Easy Access of Indices Of Alphabets
itos={ i:j for (i,j)in enumerate(s)}
stoi={itos[i]:i for i in itos.keys()}


In [4]:
print(names[:5])

['emily', 'emma', 'madison', 'olivia', 'hannah']


In [7]:
one_hot = []
max_length = max(len(name) for name in names) + 2  # +2 for start and end tokens

for name in names:
    lst = [F.one_hot(torch.tensor(0), 27)]  
    for char in name:
        lst.append(F.one_hot(torch.tensor(stoi[char]), 27))
    lst.append(F.one_hot(torch.tensor(0), 27))  

    # Pad to max_length using zero vectors
    while len(lst) < max_length:
        lst.append(torch.zeros(27))  
    
    one_hot.append(torch.stack(lst))  

one_hot = torch.stack(one_hot)  

In [12]:
labels=[]
for name in names:
    indices = [stoi[ch] for ch in name]  
    indices = indices + [0]  
    while len(indices) < max_length:
        indices.append(0)  
    labels.append(torch.tensor(indices, dtype=torch.long))
    
labels=torch.stack(labels)

In [13]:
labels.shape

torch.Size([32060, 17])

In [32]:
model=nn.Sequential(
    nn.RNN(27,32,batch_first=True),
    nn.Linear(32,27)
    
)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=0.01)

In [33]:
epochs=700
for epoch in range(epochs):
    optimizer.zero_grad()

    rnn_out, _ = model[0](one_hot) 
    outs = model[1](rnn_out)
    loss = criterion(outs.view(-1, 27), labels.view(-1))

    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')
print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')
    


Epoch 0, Loss: 3.2373
Epoch 100, Loss: 1.0102
Epoch 200, Loss: 0.9554
Epoch 300, Loss: 0.9331
Epoch 400, Loss: 0.9205
Epoch 500, Loss: 0.9116
Epoch 600, Loss: 0.9050
Epoch 700, Loss: 0.8999


In [34]:
def generate_name(model, start_char='.', max_len=10):
    model.eval()
    name = [stoi[start_char]]
    hidden = None  

    for _ in range(max_len):
        input_tensor = torch.tensor([name[-1]], dtype=torch.long)
        input_tensor = F.one_hot(input_tensor, num_classes=27).float().unsqueeze(0)  

        rnn_out, hidden = model[0](input_tensor, hidden)  
        output = model[1](rnn_out)  

        probabilities = torch.softmax(output.squeeze(), dim=-1)  # Convert to probabilities
        next_char = torch.multinomial(probabilities, num_samples=1).item()  # Sample next char

        if next_char == 0:  # End token
            break
        name.append(next_char)
    
    return ''.join(itos[i] for i in name[1:])  

# Generate different names
for _ in range(10):
    print(generate_name(model))


kandralia
sistiy
jaxobh
jeifare
smajaen
dora
jamike
javon
anday
kaider
